### _*Struct*_: A data "structure" that groups related variables under one name.
- can contain many different data types
- variables in a stuct are known as "fields" or "members"
- Type Declaration - the definition of a struct using the `struct` keyword
- Concrete Type - a type that can be instantiated (most common structs are concrete)
- Abstract Type - a type that cannot be instantiated, used for hierarchy
- Structs are immutable by default in Julia
- Constructors - methods for creating new instances

In [ ]:
struct Temperature
    kelvin::Float64
    
    # Inner constructor
    function Temperature(k::Float64)
        if k < 0
            throw(ArgumentError("Temperature cannot be below absolute zero"))
        end
        new(k)
    end
end

# Outer constructor
function Temperature(celsius::Float64, unit::String="C")
    if unit == "C"
        return Temperature(celsius + 273.15)
    elseif unit == "F"
        return Temperature((celsius - 32) * 5/9 + 273.15)
    end
end

Temperature

# Structs are useful in all kinds of ways.
___

### Example 1: Passing-in Simulation Parameters

In [4]:
# Example 1: Simulation Parameters Struct
struct SimulationParameters
    # Numerical parameters
    dt::Float64                # timestep
    total_time::Float64       # total simulation time
    tolerance::Float64        # convergence tolerance
    
    # Physical parameters
    temperature::Float64      # system temperature
    pressure::Float64        # system pressure
    volume::Float64          # system volume
    
    # Boundary conditions
    periodic::Bool           # periodic boundaries?
    boundary_type::String    # type of boundary conditions
    
    # Constructor with default values
    function SimulationParameters(;
        dt=1e-3,
        total_time=1.0,
        tolerance=1e-6,
        temperature=300.0,
        pressure=1.0,
        volume=1.0,
        periodic=true,
        boundary_type="periodic"
    )
        new(dt, total_time, tolerance, temperature, pressure, volume, periodic, boundary_type)
    end
end

In [7]:
# Usage example with simulation parameters:
function run_simulation(params::SimulationParameters)
    t = 0.0
    while t < params.total_time
        # Simulation steps here
        t += params.dt
        
        # Use parameters naturally
        if params.periodic
            # Handle periodic boundaries
        end
    end
end

# Create parameters and run
params = SimulationParameters(
    dt=0.01,
    total_time=10.0,
    temperature=350.0
)
run_simulation(params)

### Example 2: Finite Element Meshes

In [1]:
# Example 2: Finite Element Mesh Structure
struct MeshElement
    nodes::Vector{Int64}
    coordinates::Matrix{Float64}
    material_id::Int64
end

struct Mesh
    elements::Vector{MeshElement}
    node_coordinates::Matrix{Float64}
    boundary_nodes::Vector{Int64}
end

### Example 3: Parameters for a (Climate) Model

In [ ]:
# Example 4: Climate Model Parameters
struct AtmosphereParams
    num_vertical_layers::Int64
    radiation_timestep::Float64
    convection_scheme::String
    cloud_microphysics::Bool
end

struct OceanParams
    depth_levels::Vector{Float64}
    mixing_scheme::String
    enable_sea_ice::Bool
end

# You can nest structs as well, for example:
struct ClimateModel
    atmosphere::AtmosphereParams # contains AtmosphereParams struct
    ocean::OceanParams # contains OceanParams struct
    coupling_frequency::Float64
end

### Example 4: A Complicated Simulation

In [8]:
# First level structs (individual parameter groups)
struct MaterialProperties
    density::Float64
    thermal_conductivity::Float64
    specific_heat::Float64
end

struct NumericalSettings
    spatial_resolution::Float64
    time_resolution::Float64
    max_iterations::Int64
    convergence_tolerance::Float64
end

struct OutputSettings
    save_frequency::Int64
    output_directory::String
    save_fields::Vector{String}
end

# Top-level struct that contains ("nests") the other structs
struct ComplexSimulation
    material::MaterialProperties    # Contains a MaterialProperties struct
    numerics::NumericalSettings    # Contains a NumericalSettings struct
    output::OutputSettings         # Contains an OutputSettings struct
    enable_parallel::Bool
    debug_mode::Bool
end

# Example of how to create and use nested parameters:
function setup_simulation()
    # Create the individual parameter structs
    material = MaterialProperties(
        1000.0,             # density
        0.58,               # thermal_conductivity
        4184.0              # specific_heat
    )
    
    numerics = NumericalSettings(
        0.01,               # spatial_resolution
        0.001,              # time_resolution
        1000,               # max_iterations
        1e-6                # convergence_tolerance
    )
    
    output = OutputSettings(
        100,                # save_frequency
        "results/",         # output_directory
        ["temperature", "pressure"]  # save_fields
    )
    
    # Create the top-level simulation struct that contains all parameters
    sim_params = ComplexSimulation(
        material,           # nest material properties
        numerics,           # nest numerical settings
        output,            # nest output settings
        true,              # enable_parallel
        false              # debug_mode
    )
    
    return sim_params
end

# Example of using nested parameters in a simulation
function run_simulation(params::ComplexSimulation)
    # Access nested parameters using dot notation
    Δx = params.numerics.spatial_resolution
    Δt = params.numerics.time_resolution
    ρ = params.material.density
    
    # Use the parameters in calculations
    thermal_diffusivity = params.material.thermal_conductivity / 
                         (ρ * params.material.specific_heat)
    
    # Check stability criterion
    if thermal_diffusivity * Δt / (Δx^2) > 0.5
        @warn "Simulation might be unstable!"
    end
    
    # Setup output based on nested output parameters
    if !isdir(params.output.output_directory)
        mkdir(params.output.output_directory)
    end
    
    # Simulation logic would go here...
end

# Create and run the simulation
params = setup_simulation()
run_simulation(params)

# You can also access or modify individual parameters
println("Current spatial resolution: ", params.numerics.spatial_resolution)
println("Output directory: ", params.output.output_directory)
println("Material density: ", params.material.density)

Current spatial resolution: 0.01
Output directory: results/
Material density: 1000.0
